In [ ]:
import requests 
from datetime import datetime
import json
import pandas as pd
import requests

In [ ]:
def fetch_coin_dataset():
    tenx_df = pd.read_csv("./dataset/10x_coins.csv")
    not_tenx_df = pd.read_csv("./dataset/not_10x_coins.csv")

    combined_df = pd.concat([tenx_df, not_tenx_df], ignore_index=True)

    return combined_df

In [ ]:
def fetching_market_data():
    response = requests.get('https://data-api.coindesk.com/spot/v1/markets/instruments',
        params={"instrument_status":"ACTIVE","api_key":"36f52f658075058dc56593bd23430ec5ec5f3d3a8d53621c99c7c85f21df66e4"},
        headers={"Content-type":"application/json; charset=UTF-8"}
    )

    json_response = response.json()
    
    with open("./dataset/market_data.json", "w") as f:
        f.write(json.dumps(json_response))

    return json_response

In [ ]:
def get_symbol_by_name(coin_dataset,coin_name):
    coin_name = coin_name.lower()

    # Lower both sides for comparison
    result = coin_dataset[coin_dataset['name'].str.lower().str.contains(coin_name, na=False)]

    if not result.empty:
        return result.iloc[0]['symbol']
    else:
        return "Coin name not found"

In [ ]:
# Helper Function
def convert_timestamp_to_unix(time_string):
    dt = datetime.strptime(time_string, "%Y-%m-%d %H:%M:%S")

    # Convert to Unix timestamp
    unix_timestamp = int(dt.timestamp())

    # Print the Unix timestamp
    return unix_timestamp

def convert_timestamp_date(timestamp):
    utc_time = datetime.fromtimestamp(timestamp)
    return utc_time.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
def get_earliest_trade_for_coin_with_time(coin_symbol, start_timestamp, end_timestamp, market_data):
    coin_symbol = coin_symbol.lower()

    earliest_market = None  
    earliest_instrument = None  
    earliest_timestamp = float("inf")

    for market, exchange_data in market_data["Data"].items(): 
        instruments = exchange_data.get("instruments", {})

        for instrument, info in instruments.items():  
            instrument_lower = instrument.lower()

            # Check if the instrument matches the coin and has a valid suffix (USDT, USD, or USDC)
            if instrument_lower == coin_symbol + "-usdt" or instrument_lower == coin_symbol + "-usd" or instrument_lower == coin_symbol + "-usdc":
                timestamp = info.get("FIRST_TRADE_SPOT_TIMESTAMP")
                
                # Check if the timestamp is within the specified range
                if timestamp is not None and start_timestamp <= timestamp <= end_timestamp:
                    # Update earliest market if this timestamp is earlier than the current one
                    if timestamp < earliest_timestamp:
                        earliest_timestamp = timestamp
                        earliest_market = market
                        earliest_instrument = instrument

    if earliest_instrument:
        return {
            "market": earliest_market,
            "instrument": earliest_instrument,
            "timestamp": earliest_timestamp
        }
    else:
        return {
            "message": f"No matching market found for coin '{coin_symbol.upper()}' with USD, USDT, or USDC pairs within the specified time range."
        }

In [ ]:
def fetch_price_data(earliest_trade, start_unix,end_unix):
    price_data = []
    
    response = requests.get('https://data-api.coindesk.com/spot/v1/historical/hours',
        params={"market":earliest_trade["market"],"instrument":earliest_trade["instrument"],"limit":2000,"aggregate":1,"fill":"true","apply_mapping":"true","response_format":"JSON","to_ts":end_unix,"api_key":"36f52f658075058dc56593bd23430ec5ec5f3d3a8d53621c99c7c85f21df66e4"},
        headers={"Content-type":"application/json; charset=UTF-8"}
    )

    price_response = response.json()

    price_data.append(price_response)

    earliest_timestamp = price_response["Data"][0]["TIMESTAMP"]

    while True:

        if earliest_timestamp < start_unix:
            break

        response = requests.get('https://data-api.coindesk.com/spot/v1/historical/hours',
            params={"market":earliest_trade["market"],"instrument":earliest_trade["instrument"],"limit":2000,"aggregate":1,"fill":"true","apply_mapping":"true","response_format":"JSON","to_ts":earliest_timestamp,"api_key":"36f52f658075058dc56593bd23430ec5ec5f3d3a8d53621c99c7c85f21df66e4"},
            headers={"Content-type":"application/json; charset=UTF-8"}
        )

        price_response = response.json()

        price_data.append(price_response)

        # No later timestamp is available for that day
        if earliest_timestamp == price_response["Data"][0]["TIMESTAMP"]:
            earliest_timestamp = price_response["Data"][0]["TIMESTAMP"] - 86400
        else:
            earliest_timestamp = price_response["Data"][0]["TIMESTAMP"]
    
    return price_data

In [ ]:
def export_to_csv(price_data):
    desired_columns = [
        "TIMESTAMP", "OPEN", "HIGH", "LOW", "CLOSE",
        "TOTAL_TRADES", "TOTAL_TRADES_BUY", "TOTAL_TRADES_SELL", "TOTAL_TRADES_UNKNOWN",
        "VOLUME", "QUOTE_VOLUME", "VOLUME_BUY", "QUOTE_VOLUME_BUY",
        "VOLUME_SELL", "QUOTE_VOLUME_SELL", "FIRST_TRADE_PRICE", "HIGH_TRADE_PRICE",
        "LOW_TRADE_PRICE", "LAST_TRADE_PRICE"
    ]

    # Flatten and filter
    flattened_filtered_data = []
    for item in price_data:
        for row in item.get("Data", []):
            filtered_row = {key: row.get(key, None) for key in desired_columns}
            flattened_filtered_data.append(filtered_row)

    # Convert to DataFrame
    df = pd.DataFrame(flattened_filtered_data)
    df['TIMESTAMP'] = df['TIMESTAMP'].apply(convert_timestamp_date)

    df.reset_index(drop=True,inplace=True)
    df.to_csv("price_dataset.csv",index=False)

    Runner

In [ ]:
# Input
coin_name = "ai16z"
# start_time_string = "2024-10-27 00:00:00"
# end_time_string = "2025-01-05 00:00:00"
start_time_string = "2024-10-28 00:00:00"
end_time_string = "2024-12-30 00:00:00"

# Preprocessing
all_coins = fetch_coin_dataset()
market_data = fetching_market_data()

symbol = get_symbol_by_name(all_coins,coin_name)
start_unix = convert_timestamp_to_unix(start_time_string)
end_unix = convert_timestamp_to_unix(end_time_string)

# Fetching price and trade
earliest_trade = get_earliest_trade_for_coin_with_time(symbol,start_unix,end_unix,market_data)
price_data = fetch_price_data(earliest_trade,start_unix,end_unix)

print("The start date and time for price data:",convert_timestamp_date(price_data[-1]["Data"][0]["TIMESTAMP"]))
print("The end date and time for price data:",convert_timestamp_date(price_data[0]["Data"][-1]["TIMESTAMP"]))

export_to_csv(price_data)